In [ ]:
async def cmd(cbp_remote,name,parameters=[]):
    cmd_topic = getattr(cbp_remote,f"cmd_{name}").DataType()
    for parameter in parameters:
        setattr(cmd_topic,parameter[0],parameter[1])
    cmd_ack = await getattr(cbp_remote,f"cmd_{name}").start(cmd_topic,timeout=10)

In [1]:
import glob
from lsst.ts import salobj
import SALPY_FiberSpectrograph
import SALPY_EFD
import logging
import asyncio
import requests
from astropy.io import fits

Could not import lsstcppimport; please ensure the base package has been built (not just setup).



In [2]:
fiber_spectrograph_remote = salobj.Remote(SALPY_FiberSpectrograph)
efd_remote = salobj.Remote(SALPY_EFD,include=["largeFileObjectAvailable"])

In [ ]:
async def take_spectrum(imageType="test",integrationTime=0.2,lamp="kiloarc"):
    with open(f"{imageType}_{integrationTime}_{lamp}.txt","w") as file:
        fiber_spectrograph_remote.cmd_captureSpectImage.set(imageType=imageType,integrationTime=integrationTime,lamp=lamp)
        # await lfo_data = efd_remote.evt_largeFileObjectAvailable.next(timeout=10)
        spectrograph_lfo_data_coro = fiber_spectrograph_remote.evt_largeFileObjectAvailable.next(timeout=10,flush=True)
        await fiber_spectrograph_remote.cmd_captureSpectImage.start(timeout=10)
        spectrograph_lfo_data = await spectrograph_lfo_data_coro
        file.write(f"{spectrograph_lfo_data.url}")
    with open(f"{imageType}_{integrationTime}_{lamp}.txt","r") as file:
        for line in file:
            line_split = line.split('/')
            url = requests.get(line)
            with open(f"{line_split[-1]}","wb") as fits_file:
                fits_file.write(url.content)
        fits_file_list = glob.glob('*.fits')
        

In [ ]:
await take_spectrum()